<a href="https://colab.research.google.com/github/Suresh045/TNSDC/blob/main/Feature_Selection_regression__rfe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt

In [ ]:
# Read the preprocessed CSV file
dataset = pd.read_csv('flightprice.csv')

# Inspect the DataFrame
print(dataset.head())

   duration  days_left   airline source_city departure_time  stops  \
0       223          4    IndiGo     Kolkata      Afternoon      0   
1       249         29     GoAir       Delhi          Night      0   
2       119         17     GoAir       Delhi      Afternoon      0   
3       131         26    IndiGo      Mumbai        Evening      0   
4        86          3  SpiceJet       Delhi        Evening      0   

  arrival_time destination_city     class  price  
0    Afternoon        Bangalore   Economy  14087  
1      Morning          Kolkata   Economy   6582  
2        Night          Kolkata  Business  12654  
3      Evening        Hyderabad   Economy   8514  
4      Evening          Chennai  Business  11785  


In [ ]:
dataset.columns

Index(['duration', 'days_left', 'airline', 'source_city', 'departure_time',
       'stops', 'arrival_time', 'destination_city', 'class', 'price'],
      dtype='object')

In [ ]:
indep_X=dataset[['duration', 'days_left', 'airline', 'source_city',
       'departure_time', 'stops', 'arrival_time', 'destination_city', 'class']]

In [ ]:
dep_Y=dataset[['price']]

# Feature Selection

# Recursive Feature Elimination

In [ ]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)

        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        return X_train, X_test, y_train, y_test

In [ ]:
def r2_prediction(regressor,X_test,y_test):
    y_pred = regressor.predict(X_test)
    from sklearn.metrics import r2_score
    r2=r2_score(y_test,y_pred)
    return r2

In [ ]:
def Linear(X_train,y_train,X_test):
        from sklearn.linear_model import LinearRegression
        regressor = LinearRegression()
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2

In [ ]:
def Decision(X_train,y_train,X_test):
        from sklearn.tree import DecisionTreeRegressor
        regressor = DecisionTreeRegressor(random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2

In [ ]:
def random(X_train,y_train,X_test):
        from sklearn.ensemble import RandomForestRegressor
        regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2

In [ ]:
def xgboost(X_train,y_train,X_test):
        from xgboost import XGBRegressor
        regressor = XGBRegressor(n_jobs=5,learning_rate=0.1,max_depth=10,random_state=1)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return r2

In [ ]:
def rfeFeature(indep_X, dep_Y, n):
    rfelist = []
    colnames_list = []
    # List to store column names for each model
    r2_values = []
    # List to store R2 values for each model

    # Apply one-hot encoding to categorical features
    indep_X_encoded = pd.get_dummies(indep_X, columns=['airline', 'source_city', 'departure_time', 'arrival_time', 'destination_city', 'class'], drop_first=True)

    from sklearn.linear_model import LinearRegression
    lin = LinearRegression()

    from sklearn.tree import DecisionTreeRegressor
    dec = DecisionTreeRegressor(random_state=0)

    from sklearn.ensemble import RandomForestRegressor
    rf = RandomForestRegressor(n_estimators=10, random_state=0)

    from xgboost import XGBRegressor
    xgb = XGBRegressor(n_jobs=5, learning_rate=0.1, max_depth=10, random_state=1)

    rfemodellist = [lin, dec, rf, xgb]

    for model in rfemodellist:
        log_rfe = RFE(estimator=model, n_features_to_select=n)
        log_fit = log_rfe.fit(indep_X_encoded, dep_Y)  # Use encoded features
        log_rfe_feature = log_fit.transform(indep_X_encoded) # Use encoded features
        rfelist.append(log_rfe_feature)

        # Get the column names selected by RFE
        selected_columns = [col for col, selected in zip(indep_X_encoded.columns, log_rfe.support_) if selected] # Use encoded columns
        colnames_list.append(selected_columns)

        # Fit the model and calculate and store the R2 value
        X_train, X_test, y_train, y_test = split_scalar(pd.DataFrame(log_rfe_feature), dep_Y)
        model.fit(X_train, y_train)  # Fit the model
        r2 = r2_prediction(model, X_test, y_test)
        r2_values.append(r2)

    return rfelist, colnames_list, r2_values

# Call the function with your data
rfelist, colnames_list, r2_values = rfeFeature(indep_X, dep_Y, 5)

# Print the selected column names and R2 values for each model
for model_name, selected_columns, r2_value in zip(["Linear", "Decision", "Random", "XGBoost"], colnames_list, r2_values):
    print(f"Model: {model_name}")
    print("Selected Columns:", selected_columns)
    print(f"R2 Value: {r2_value}\n")

/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example usi

Model: Linear
Selected Columns: ['airline_IndiGo', 'airline_SpiceJet', 'source_city_Hyderabad', 'departure_time_Morning', 'destination_city_Kolkata']
R2 Value: -0.06962412398920526

Model: Decision
Selected Columns: ['duration', 'days_left', 'stops', 'arrival_time_Night', 'destination_city_Delhi']
R2 Value: -0.9602117955056462

Model: Random
Selected Columns: ['duration', 'days_left', 'source_city_Hyderabad', 'departure_time_Morning', 'destination_city_Kolkata']
R2 Value: -0.08023954981877979

Model: XGBoost
Selected Columns: ['airline_SpiceJet', 'source_city_Hyderabad', 'departure_time_Morning', 'destination_city_Delhi', 'destination_city_Kolkata']
R2 Value: 0.018456995487213135



# Model Creation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import pandas as pd

# Apply one-hot encoding to categorical features
indep_X_encoded = pd.get_dummies(indep_X, columns=['airline', 'source_city', 'departure_time', 'arrival_time', 'destination_city', 'class'], drop_first=True)

x_train, x_test, y_train, y_test = train_test_split(indep_X_encoded, dep_Y, test_size=0.30, random_state=0)

regressor_dt = DecisionTreeRegressor(criterion='squared_error', splitter='random')
regressor_dt = regressor_dt.fit(x_train, y_train)

In [ ]:
y_pred=regressor_dt.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score
r_score=r2_score(y_test,y_pred)

In [ ]:
r_score

0.9726866831886678

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
regressor_gbr = GradientBoostingRegressor(n_estimators=500,max_depth=4,min_samples_split=5,learning_rate=0.01,loss="squared_error")
regressor_gbr.fit(x_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/ensemble/_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


GradientBoostingRegressor(learning_rate=0.01, max_depth=4, min_samples_split=5,
                          n_estimators=500)

In [ ]:
y_pred=regressor_gbr.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score
r_score=r2_score(y_test,y_pred)
r_score

-0.0536463453093865

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor_rf = RandomForestRegressor(n_estimators=100,random_state=0)
regressor_rf.fit(x_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(random_state=0)

In [ ]:
y_pred=regressor_rf.predict(x_test)

In [ ]:
r_score

-0.0536463453093865

In [ ]:
# When comparing, Decision Tree algorithm gives maximum accuracy of 97%

In [ ]:
import pickle
Finalised_Model="Finalized_model.sav"

In [ ]:
pickle.dump(regressor_dt,open(Finalised_Model,'wb'))